# Outlier correction

We import pose-estimation data, and perform outlier correction on that pose-estimation data using heuristic rules. The application of those rules are described here:
https://github.com/sgoldenlab/simba/blob/master/misc/Outlier_settings.pdf

In [1]:
from simba.outlier_tools.outlier_corrector_movement import OutlierCorrecterMovement
from simba.outlier_tools.outlier_corrector_location import OutlierCorrecterLocation
from simba.utils.cli.cli_tools import set_outlier_correction_criteria_cli
from simba.pose_importers.dlc_importer_csv import import_dlc_csv_data

In [2]:
# DEFINITIONS: HERE WE (i) DEFINE THE PATHS TO OUR DATA / SIMBA PROJECT, (ii) HOW WE SHOULD INTERPOLATE / SMOOTH OUR NEW
# DATA, AND (III) THE ATTRIBUTES OF OUR NEW VIDEOS (FPS ETC.)

## Define the path to our SimBA project config ini
CONFIG_PATH = r"C:\troubleshooting\two_black_animals_14bp\project_folder\project_config.ini"

## Define the path to the directory holding our new DLC CSV pose-estimation data
DATA_DIR = r"C:\troubleshooting\two_black_animals_14bp\dlc_data"

## Define if / how you want to interpolate missing pose-estimation data,
## and if/how you want to smooth the new pose estimation data: here we do neither.
INTERPOLATION_SETTING = None # EXAMPLE OPTIONS: {'type': 'body-parts', 'method': 'nearest'}, {'type': 'animals', 'method': 'linear'}, {'type': 'animals', 'method': 'quadratic'}, {'type': 'body-parts', 'method': 'linear'}
SMOOTHING_SETTING = None # EXAMPLE OPTIONS: {'time_window': 500, 'method': 'savitzky-golay'}, {'time_window': 500, 'method': 'gaussian'}

## Define the fps and the pixels per millimeter of the incoming data: has to be the same for all new videos.
## if you have varying fps / px per millimeter / resolutions, then use gui (2023/05)
FPS = 15
PX_PER_MM = 4.6
RESOLUTION = (600, 400) # WIDTH X HEIGHT

#Define the body-parts and critera we want to use for outlier correction. NOTE: You can also set this manually in the project_config.ini or thrugh
# the SimBA GUI.
AGGREGATION_METHOD = 'mean'
BODY_PARTS = {'Animal_1': {'Movement': ['Nose_1', 'Tail_base_1'],
                           'Location': ['Nose_1', 'Tail_base_1']},
              'Animal_2': {'Movement': ['Nose_2', 'Tail_base_2'],
                           'Location': ['Nose_2', 'Tail_base_2']}}
MOVEMENT_CRITERION = 0.7
LOCATION_CRITERION = 2.0

In [3]:
# WE RUN THE DATA IMPORTER FOR OUR DIRECTORY OF FILES
## This imports your DLC files in the ``DATA_DIR`` according to the smoothing / interpolation settings defined above

import_dlc_csv_data(config_path=CONFIG_PATH,
                    interpolation_settings=INTERPOLATION_SETTING,
                    smoothing_settings=SMOOTHING_SETTING,
                    data_path=DATA_DIR)

Importing Test_1 to SimBA project...
Pose-estimation data for video Test_1 imported to SimBA project (elapsed time: 0.0546s)...
Importing Test_2 to SimBA project...
Pose-estimation data for video Test_2 imported to SimBA project (elapsed time: 0.052s)...
Importing Test_3 to SimBA project...
Pose-estimation data for video Test_3 imported to SimBA project (elapsed time: 0.0463s)...
Importing Test_4 to SimBA project...
Pose-estimation data for video Test_4 imported to SimBA project (elapsed time: 0.0431s)...
Importing Test_5 to SimBA project...
Pose-estimation data for video Test_5 imported to SimBA project (elapsed time: 0.0482s)...
Importing Test_6 to SimBA project...
Pose-estimation data for video Test_6 imported to SimBA project (elapsed time: 0.0583s)...
Importing Test_7 to SimBA project...
Pose-estimation data for video Test_7 imported to SimBA project (elapsed time: 0.0503s)...
Importing Test_8 to SimBA project...
Pose-estimation data for video Test_8 imported to SimBA project (ela

In [5]:
#We set the outlier criteria in the project_config.ini NOTE: You can also set this manually in the project_config.ini or thrugh
# the SimBA GUI. If this has already been done, there is **no need** to call `set_outlier_correction_criteria_cli`.
set_outlier_correction_criteria_cli(config_path=CONFIG_PATH,
                                    aggregation=AGGREGATION_METHOD,
                                    body_parts=BODY_PARTS,
                                    movement_criterion=MOVEMENT_CRITERION,
                                    location_criterion=LOCATION_CRITERION)

SIMBA COMPLETE: Outlier parameters set (elapsed time: 0.003s) 	complete


In [6]:
# Finally, we run the outlier correction (NOTE: SEE CELL BELOW FOR ALTERNATIVE WAY OF RUNNING OUTLIER CORRECTION ACROSS MULTIPLE CORES)
_ = OutlierCorrecterMovement(config_path=CONFIG_PATH).run()
_ = OutlierCorrecterLocation(config_path=CONFIG_PATH).run()

Processing video Test_1. Video 1/9...
Corrected movement outliers for file Test_1 (elapsed time: 0.435s)...
Processing video Test_2. Video 2/9...
Corrected movement outliers for file Test_2 (elapsed time: 0.1166s)...
Processing video Test_3. Video 3/9...
Corrected movement outliers for file Test_3 (elapsed time: 0.1158s)...
Processing video Test_4. Video 4/9...
Corrected movement outliers for file Test_4 (elapsed time: 0.114s)...
Processing video Test_5. Video 5/9...
Corrected movement outliers for file Test_5 (elapsed time: 0.1183s)...
Processing video Test_6. Video 6/9...
Corrected movement outliers for file Test_6 (elapsed time: 0.1124s)...
Processing video Test_7. Video 7/9...
Corrected movement outliers for file Test_7 (elapsed time: 0.1151s)...
Processing video Test_8. Video 8/9...
Corrected movement outliers for file Test_8 (elapsed time: 0.117s)...
Processing video Test_9. Video 9/9...
Corrected movement outliers for file Test_9 (elapsed time: 0.1152s)...
SIMBA COMPLETE: Log fo

In [7]:
# OPTIONAL: If you find that the outlier correction - as run in the immediate above cell - is slow, we could run outlier 
# correction over multiple cores. If you choose this approach, make sure you are running the latest version of SimBA.
# You can update SimBA by running `pip install simba-uw-tf-dev --upgrade`

from simba.outlier_tools.outlier_corrector_location_mp import OutlierCorrecterLocationMultiprocess
from simba.outlier_tools.outlier_corrector_movement_mp import OutlierCorrecterMovementMultiProcess

CORE_COUNT = -1 #SPECIFY HOW MANY CPU CORES YOU WANT TO USE. PASS -1 TO USE ALL AVAILABLE CORES. E.g., PASS 8 TO USE 8 CPU CORES.


_ = OutlierCorrecterMovementMultiProcess(config_path=CONFIG_PATH, core_cnt=CORE_COUNT).run()
_ = OutlierCorrecterLocationMultiprocess(config_path=CONFIG_PATH, core_cnt=CORE_COUNT).run()

Video Test_1 complete...
Video Test_2 complete...
Video Test_3 complete...
Video Test_4 complete...
Video Test_5 complete...
Video Test_6 complete...
Video Test_7 complete...
Video Test_8 complete...
Video Test_9 complete...
SIMBA COMPLETE: Log for corrected "movement outliers" saved in C:\troubleshooting\two_black_animals_14bp\project_folder\logs (elapsed time: 4.2945s) 	complete
Video Test_1 complete...
Video Test_2 complete...
Video Test_3 complete...
Video Test_4 complete...
Video Test_5 complete...
Video Test_6 complete...
Video Test_7 complete...
Video Test_8 complete...
Video Test_9 complete...
SIMBA COMPLETE: Log for corrected "location outliers" saved in project_folder/logs (elapsed time: 4.0219s) 	complete
